In [1]:
import utils
from utils import *

trainPath=r"C:\Users\sivateja_k\Desktop\NLP_Project\dataset\Reviews.csv"
review,sentiment,df=data_preprocess(trainPath)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sivateja_k\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


resulting filtered data length after dropping rating 3 should be 525814
Filtered data length is 525814


C:\Users\sivateja_k\Desktop\NLP_Project\utils.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['sentiment']=filtered_data['Score'].apply(assign_score)
C:\Users\sivateja_k\Anaconda3\envs\nlpproject\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [2]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten

review_train,review_test,target_train,target_test=train_test_split(review,sentiment,test_size=0.2,random_state=0)
top_words = 6000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(review_train)
list_tokenized_train = tokenizer.texts_to_sequences(review_train)

max_review_length = 130
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = target_train

Using TensorFlow backend.
C:\Users\sivateja_k\Anaconda3\envs\nlpproject\lib\site-packages\tensorflow\python\framework\dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sivateja_k\Anaconda3\envs\nlpproject\lib\site-packages\tensorflow\python\framework\dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sivateja_k\Anaconda3\envs\nlpproject\lib\site-packages\tensorflow\python\framework\dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sivateja

In [3]:
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 32)           192032    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 245,333.0
Trainable params: 245,333
Non-trainable params: 0.0
_________________________________________________________________


In [4]:
model.fit(X_train,y_train, epochs=10, batch_size=64, validation_split=0.2)

Train on 233061 samples, validate on 58266 samples
Epoch 1/10
233061/233061 [==============================] - 932s - loss: 0.2199 - acc: 0.9118 - val_loss: 0.1898 - val_acc: 0.9245
Epoch 2/10
233061/233061 [==============================] - 957s - loss: 0.1745 - acc: 0.9310 - val_loss: 0.1747 - val_acc: 0.9313
Epoch 3/10
233061/233061 [==============================] - 858s - loss: 0.1548 - acc: 0.9392 - val_loss: 0.1707 - val_acc: 0.9335
Epoch 4/10
233061/233061 [==============================] - 983s - loss: 0.1418 - acc: 0.9448 - val_loss: 0.1786 - val_acc: 0.9319
Epoch 5/10
233061/233061 [==============================] - 878s - loss: 0.1296 - acc: 0.9500 - val_loss: 0.1800 - val_acc: 0.9338
Epoch 6/10
233061/233061 [==============================] - 850s - loss: 0.1172 - acc: 0.9552 - val_loss: 0.1823 - val_acc: 0.9318
Epoch 7/10
233061/233061 [==============================] - 878s - loss: 0.1049 - acc: 0.9601 - val_loss: 0.1955 - val_acc: 0.9309
Epoch 8/10
233061/233061 [======

In [5]:
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix
list_tokenized_test = tokenizer.texts_to_sequences(review_test)
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = target_test
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.9276828866432337
F1-score:  0.9574902543159458
Confusion matrix:


array([[ 8248,  3186],
       [ 2081, 59317]], dtype=int64)